# ゼロから作る Deep Learning 4 強化学習編 勉強ノート 第2章 〜マルコフ決定過程〜

ここでは、エージェントの行動によって状況が変わる問題の一部で定式化される「<font color="red">**マルコフ決定過程**</font>」について扱う。

## マルコフ決定過程(MDP)とは

マルコフ決定過程(Markov Decision Process)とは、「エージェントが、環境と相互作用しながら行動を決定する過程」である。

### 具体例(*コーディング*)

グリッドで区切られた世界を「グリッドワールド」と呼ぶことにする。  
エージェントは最初スタート地点におり、そこからできる限り多くのポイントを獲得してゴールに向かうものとする。  
エージェントはグリッドワールド内を上下左右に1マスずつ動くことができるが、障害物を通り抜けることはできないとする。  
以下にグリッドワールドの一例を作成する。

In [ ]:
# ライブラリのインポート
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# グリッドワールドを作成する関数を定義
# 0: 進める道, 1: 障害物, 2: スタート, 3: ゴール, 4: 加点ポイント, 5: 減点ポイント
def create_grid_world(
        rows: int, cols: int,
        start: set, goal: set,
        obstacles: list, items: dict, enemies: dict
    ):
    grid = np.zeros((rows, cols))
    points = np.zeros((rows, cols), dtype=int)

    for obs in obstacles:
        grid[obs] = 1

    grid[start] = 2
    if goal != ():
        grid[goal] = 3

    for item, point in items.items():
        grid[item] = 4
        points[item] = point

    for enemy, point in enemies.items():
        grid[enemy] = 5
        points[enemy] = point

    return grid, points


# グリッドワールドを描画する関数を定義
def draw_grid_world(grid, points, figsize_x, figsize_y):
    rows, cols = grid.shape
    fig = plt.figure(figsize=(figsize_x, figsize_y), tight_layout=True)
    ax = fig.subplots()

    ax.matshow(grid, vmin=-0.5, vmax=5)

    ax.set_xticks(np.arange(-.5, cols, 1), minor=True)
    ax.set_yticks(np.arange(-.5, rows, 1), minor=True)
    ax.grid(which="minor", color="black", linestyle="-", linewidth=2)

    for (i, j), val in np.ndenumerate(grid):
        if val == 1:
            ax.text(
                j, i, "X", ha="center", va="center",
                color="yellow", fontsize=16, fontweight="bold"
            )
        elif val == 2:
            ax.text(
                j, i, "S", ha="center", va="center",
                color="red", fontsize=16, fontweight="bold"
            )
        elif val == 3:
            ax.text(
                j, i, "G", ha="center", va="center",
                color="blue", fontsize=16, fontweight="bold"
            )
        elif val == 4:
            ax.text(
                j, i, f"+{points[i, j]}", ha="center", va="center",
                color="white", fontsize=16, fontweight="bold"
            )
        elif val == 5:
            ax.text(
                j, i, f"{points[i, j]}", ha="center", va="center",
                color="black", fontsize=16, fontweight="bold"
            )

    ax.set_xticks([])
    ax.set_yticks([])

    plt.show()

In [ ]:
# グリッドワールドのパラメータを設定
rows, cols = 10, 10
start = (0, 0)
goal = (9, 9)
obstacles = [(1, 1), (1, 2), (2, 1), (3, 2), (7, 6), (7, 8)]
items = {(1, 3): +5, (2, 2): +3, (3, 4): +5, (5, 6): +2, (8, 6): +3}
enemies = {(3, 5): -4, (7, 7): -2, (9, 7): -3}

# グリッドワールドを作成して描画
grid, points = create_grid_world(
    rows, cols, start, goal, obstacles, items, enemies
)
draw_grid_world(grid, points, cols, rows)

スタートからゴールまで行く方法は無限にあるが、どこをどのように通るかで得られるポイントが変わってくる。  
すなわち、エージェントの行動によって、そのエージェントが置かれている状況が変わってくるということである。  
この状況のことを「<font color="red">**状態**</font>(State)」と呼ぶ。  
MDP では「時間」の概念が必要になり、その単位を「<font color="red">**タイムステップ**</font>」などと言う。

### エージェントと環境の相互作用

MDP ではエージェントと環境の間で相互にやり取りをする。  
重要なのは、エージェントが行動を起こすことで状態が遷移するということである。  
それに伴い、得られる報酬も変わってくる。

![MDP](https://assets.st-note.com/production/uploads/images/15214525/rectangle_large_type_2_4bf7be1ec2e8476ebcba33684604ae5b.png?width=1200)

(出典: https://note.com/npaka/n/n7fca1d4d5ce8)

上図において、時刻 $t$ で報酬 $R_t$ を受け取って状態が $S_t$ となり、この状態に基づいてエージェントが行動 $A_t$ を行い、報酬 $R_{t+1}$ を受け取って状態が $S_{t+1}$ へと遷移する。  
すなわち、このエージェントと環境の相互作用は次のような遷移を生む。

$$S_0, A_0, R_1, S_1, A_1, R_2, S_2, A_2, R_3 \cdots$$

強化学習の分野では、報酬 $R_t$ の時間のタイミングについて、

- 状態 $S_t$ で行動 $A_t$ を行い、報酬 $R_t$ を受け取り、次の状態 $S_{t+1}$ に遷移する
- 状態 $S_t$ で行動 $A_t$ を行い、報酬 $R_{t+1}$ を受け取り、次の状態 $S_{t+1}$ に遷移する

の2パターンの慣例が見られるが、以降プログラミングとの親和性を考慮して前者を採用する。

## 環境とエージェントの定式化

MDP は、エージェントと環境とのやり取りを数式で定式化する。

- 状態遷移: 状態はどのように遷移するか
- 報酬: 報酬はどのように与えられるか
- 方策: エージェントはどのように行動を決定するか

### 状態遷移

上記のグリッドワールドのスタート地点においては、右または下にしか動くことができない。  
仮に右に動く確率を0.3、下に動く確率を0.7とすると次の状態 $s'$ に遷移するのは、今の状態 $s$ と行動 $a$ によって次のように表記できる。

\begin{eqnarray*}
p(s'|s, a=\text{Right}) &=& 0.3 \\
p(s'|s, a=\text{Down}) &=& 0.7
\end{eqnarray*}

この $p(\cdot)$ は<font color="red">**状態遷移確率**</font>(State Transition Probability)と呼ばれる。  
$p(s'|s, a)$ は現在の状態 $s$ および 行動 $a$ にのみ依存しているので、これまでにどのような状態にあって、どのような行動を行ってきたかという過去の情報は不要である。  
このような性質のことを<font color="red">**マルコフ性**</font>(Markov Property)という。

### 報酬関数

エージェントが状態 $s$ にいて、行動 $a$ を行い、状態が $s'$ に遷移した時に得られる報酬の関数を $r(s, a, s')$ と定義する。  
これを<font color="red">**報酬関数**</font>(Reward Function)という。  
例えば、エージェントが(0, 3)の位置にいたとすると、

\begin{eqnarray*}
r(s=(0, 3), a=\text{Right}, s'=(0, 4)) &=& 0 \\
r(s=(0, 3), a=\text{Left}, s'=(0, 2)) &=& 0 \\
r(s=(0, 3), a=\text{Down}, s'=(1, 3)) &=& 5
\end{eqnarray*}

ということになる。  
今回の場合、移動先によって報酬が決定するため、単に報酬関数を $r(s')$ とすることもできる。

### エージェントの方策

エージェントがどのように行動するかを決めるものを<font color="red">**方策**</font>(Policy)という。  
方策について重要なのは、環境がマルコフ性に基づいて遷移するため、エージェントは「現在の状態」にのみ基づいて行動が決定されるということである。  
方策は、一般的に確率的に決まる。  
先ほどの例で言えば、(0, 3)にいるとして、エージェントの行動が確率的に決まる方策 $\pi(\cdot)$ は、次のように表される。

\begin{eqnarray*}
\pi(a=\text{Right}|s=(0, 3)) &=& 0.6 \\
\pi(a=\text{Left}|s=(0, 3)) &=& 0.1 \\
\pi(a=\text{Down}|s=(0, 3)) &=& 0.3
\end{eqnarray*}

## MDP の目標

今までのことをまとめると、

- エージェントは方策 $\pi(a|s)$によって行動する
- その行動 $a$ と状態遷移確率 $p(s'|s, a)$ によって次の状態に遷移する
- 報酬関数 $r(s, a, s')$ に従って報酬を得る

となる。  

MDP の目標は、この枠組みの中で<font color="red">**最適方策**</font>(Optimal Policy)を見つけることである。  
これを定式化する上で、まずは MDP の問題が大きく

- エピソードタスク
- 連続タスク

の2つに分けられることを説明する。

### エピソードタスクと連続タスク

囲碁のように、「終わり」がある問題をエピソードタスクという。  
また、この中で始まりから終わりまでの一連の試行を「エピソード」と呼ぶ。

一方で、「終わり」がない問題のことを連続タスクという。  
例えば、在庫管理を行う問題では、エージェントがどれだけ商品を仕入れるかを判断し、売れ行きや在庫の量に応じてベストな仕入れ量を判断する。  
このような問題では、「終わり」を設けずに永遠に続く問題として定義することができる。

### 収益

時刻 $t$ において状態が $S_t$ である場合を仮定する。  
そしてエージェントが方策 $\pi$ によって行動 $A_t$ を行い、報酬 $R_t$ を得て新しい状態 $S_{t+1}$ に遷移するとする。  
ここで、<font color="red">**収益**</font>(Return) $G_t$ を次のように定義する。

$$G_t = R_t + \gamma R_{t+1} + \gamma^2 R_{t+2} + \cdots$$

時間が進むにつれて $\gamma$ によって指数関数的に報酬が減衰する。  
この $\gamma$ を<font color="red">**割引率**</font>(Discount Rate)と呼び、0.0から1.0の間の実数を設定する。  
割引率を導入する理由としては、連続タスクを想定したときに、収益が無限大になることを防ぐためである。

### 状態価値関数

エージェントと環境は「確率的に」振る舞う可能性があるため、例えばあるエピソードでは収益が10.4で、別のエピソードでは8.7といったように、同じ状態からスタートしても得られる収益はエピソードごとに変動する。  
このような確率的な挙動に対応するには、「収益の期待値」を指標とする必要がある。  
収益の期待値は次の式で表される。

$$v_\pi(s) = \mathbb{E}[G_t|S_t = s, \pi] = \mathbb{E}_\pi[G_t|S_t = s]$$

これを<font color="red">**状態価値関数**</font>(State-Value Function)という。  
方策 $\pi$ が変わればエージェントが得る報酬も変わり、その総和である収益も変わるため、方策 $\pi$ は状態価値関数の条件として与えられる。

### 最適方策と最適状態価値関数

強化学習において目標とすることは、最適方策を手に入れることである。  
ここに2つの方策 $\pi, \pi'$ があり、2つの方策において状態価値関数 $v_\pi(s), v_{\pi'}(s)$ がそれぞれ決まるとする。  
どちらの方策が優れているかを考えるためには、全ての状態において $v_\pi(s) \geq v_{\pi'}(s)$ または $v_\pi(s) \leq v_{\pi'}(s)$ を満たさなければならない。  
もし、ある状態においてはその不等号が逆転するといったことがあれば、その2つの方策に優劣をつけることはできないということである。

では、この考え方を推し進め、最適方策を $\pi_*$ と表すと、方策 $\pi_*$ は他のどの方策と比較しても、全ての状態で $v_{\pi_*}(s)$ の値が大きいと言うことができる。  
重要な事実として、 MDP では最適方策が少なくとも1つは存在し、その最適方策は「決定論的方策」である(証明略)。  
この最適方策における状態価値関数は<font color="red">**最適状態価値関数**</font>(Optimal State-Value Function)と呼ばれ、 $v_*$ で表記することにする。

## MDP の例

MDP についての具体的な問題を見ていく。

### 問題設定(*コーディング*)

2マスのグリッドワールドを考える。

- エージェントは右か左かのどちらかに移動できる
- 状態遷移は決定論的とする(エージェントが右に進む行動を行うと、エージェントの次の状態は必ず右側に遷移する)
- エージェントが(0, 1)から(0, 2)に移動したときに $+1$ の報酬を得る
- エージェントが(0, 2)から(0, 1)に移動するときのみ報酬が再度出現する
- (0, 0)と(0, 3)に行くと $-1$ の報酬を得て、それぞれ(0, 1)と(0, 2)に戻る
- 連続タスクとして考える

In [ ]:
# グリッドワールドのパラメータを設定
rows, cols = 1, 4
start = (0, 1)
goal = ()
obstacles = []
items = {(0, 2): +1}
enemies = {(0, 0): -1, (0, 3): -1}

# グリッドワールドを作成して描画
grid, points = create_grid_world(
    rows, cols, start, goal, obstacles, items, enemies
)
draw_grid_world(grid, points, cols, rows)

### 最適方策を見つける(*コーディング*)

この問題における方策は、次の4通りである。

|  | (0, 1) | (0, 2) |
| :--: | :--: | :--: |
| $\pi_1(s)$ | Right | Right |
| $\pi_2(s)$ | Right | Left |
| $\pi_3(s)$ | Left | Right |
| $\pi_4(s)$ | Left | Left |

それぞれの方策について、割引率を0.9として計算してみる。

方策 $\pi_1$ では状態価値関数はそれぞれ以下のようになる。

---

\begin{eqnarray*}
v_{\pi_1}(s = (0, 1)) &=& 1 + 0.9 \cdot (-1) + 0.9^2 \cdot (-1) + \cdots \\
&=& 1 - 0.9(1 + 0.9 + 0.9^2 + \cdots) \\
&=& 1 - \frac{0.9}{1-0.9} \\
&=& -8
\end{eqnarray*}

---

\begin{eqnarray*}
v_{\pi_1}(s = (0, 2)) &=& -1 + 0.9 \cdot (-1) + 0.9^2 \cdot (-1) + \cdots \\
&=& -1 - 0.9(1 + 0.9 + 0.9^2 + \cdots) \\
&=& -1 - \frac{0.9}{1-0.9} \\
&=& -10
\end{eqnarray*}

---

方策 $\pi_2$ では状態価値関数はそれぞれ以下のようになる。

---

\begin{eqnarray*}
v_{\pi_2}(s = (0, 1)) &=& 1 + 0.9 \cdot 0 + 0.9^2 \cdot 1 + \cdots \\
&=& 1 + 0.9^2 + 0.9^4 + \cdots \\
&=& \frac{1}{1-0.9^2} \\
&=& 5.26316 ...
\end{eqnarray*}

---

\begin{eqnarray*}
v_{\pi_2}(s = (0, 2)) &=& 0 + 0.9 \cdot 1 + 0.9^2 \cdot 0 + \cdots \\
&=& 0.9(1 + 0.9^2 + 0.9^4 + \cdots) \\
&=& \frac{0.9}{1-0.9^2} \\
&=& 4.73684 ...
\end{eqnarray*}

---

方策 $\pi_3$ では状態価値関数はそれぞれ以下のようになる。

---

\begin{eqnarray*}
v_{\pi_3}(s = (0, 1)) &=& -1 + 0.9 \cdot (-1) + 0.9^2 \cdot (-1) + \cdots \\
&=& -1 - 0.9(1 + 0.9 + 0.9^2 + \cdots) \\
&=& -1 - \frac{0.9}{1-0.9} \\
&=& -10
\end{eqnarray*}

---

\begin{eqnarray*}
v_{\pi_3}(s = (0, 2)) &=& -1 + 0.9 \cdot (-1) + 0.9^2 \cdot (-1) + \cdots \\
&=& -1 - 0.9(1 + 0.9 + 0.9^2 + \cdots) \\
&=& -1 - \frac{0.9}{1-0.9} \\
&=& -10
\end{eqnarray*}

---

方策 $\pi_4$ では状態価値関数はそれぞれ以下のようになる。

---

\begin{eqnarray*}
v_{\pi_4}(s = (0, 1)) &=& -1 + 0.9 \cdot (-1) + 0.9^2 \cdot (-1) + \cdots \\
&=& -1 - 0.9(1 + 0.9 + 0.9^2 + \cdots) \\
&=& -1 - \frac{0.9}{1-0.9} \\
&=& -10
\end{eqnarray*}

---

\begin{eqnarray*}
v_{\pi_4}(s = (0, 2)) &=& 0 + 0.9 \cdot (-1) + 0.9^2 \cdot (-1) + \cdots \\
&=& -0.9(1 + 0.9 + 0.9^2 + \cdots) \\
&=& - \frac{0.9}{1-0.9} \\
&=& -9
\end{eqnarray*}

---

In [ ]:
# 状態のラベルとそれぞれの方策の状態価値関数の値
x = ["(0, 1)", "(0, 2)"]
y_1 = [-8, -10]
y_2 = [5.26316, 4.73684]
y_3 = [-10, -10]
y_4 = [-10, -9]

# 画像のサイズを指定
plt.figure(figsize=(10, 6))

# 画像のラベル情報付与
plt.title("Each Policy Comparison", size=15, color="red")
plt.xlabel("State")
plt.ylabel("Value")

# 罫線
plt.grid()

# 4種類のグラフをプロット
plt.plot(x, y_1, label="Policy 1")
plt.plot(x, y_2, label="Policy 2")
plt.plot(x, y_3, label="Policy 3")
plt.plot(x, y_4, label="Policy 4")

# 2つのグラフの凡例を付与
plt.legend(bbox_to_anchor=(1.01, 1), loc="upper left", borderaxespad=0.)

# グラフの描画
plt.show()

以上により、方策 $\pi_2$ が全ての状態において他の方策より状態価値関数の値が大きいことがわかった。